#### 第十二章 继承的优缺点
##### 12.1 子类化内置类型很麻烦
- 内置类型的方法不会调用子类覆盖的方法。
- 直接子类化内置类型（如 dict、list 或 str）容易出错，因为内置类型的方法通常会忽略用户覆盖的方法。不要子类化内置类型，用户自己定义的类应该继承 collections 模块（http://docs.python.org/3/library/collections.html）中的类，例如: UserDict、UserList 和 UserString，这些类做了特殊设计，因此易于扩展。

In [29]:
# 内置类型dict的__init__和__update__方法会忽略我们覆盖的__setitem__方法
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [30]:
dd = DoppelDict(one=1)
print(dd)
dd['two'] = 2
print(dd)
dd.update(three=3)
print(dd)

{'one': 1}
{'one': 1, 'two': [2, 2]}
{'one': 1, 'two': [2, 2], 'three': 3}


In [31]:
# dict.update方法会忽略AnswerDict.__getitem__方法
class AnswerDict(dict):
    def __getitem__(self, key):
        return 42

In [32]:
ad = AnswerDict(a='foo')
print(ad['a'])
d = {}
d.update(ad)
print(d)

42
{'a': 'foo'}


In [33]:
import collections

class DoppelDict2(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [36]:
dd = DoppelDict2(one=1)
print(dd)
dd['two'] = 2
print(dd)
dd.update(three=3)
print(dd)

{'one': [1, 1]}
{'one': [1, 1], 'two': [2, 2]}
{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}


In [37]:
class AnswerDict2(collections.UserDict):
    def __getitem__(self, key):
        return 42

In [40]:
ad = AnswerDict2(a='foo')
print(ad['a'])
dd = {}
dd.update(ad)
print(dd)

42
{'a': 42}


##### 12.2 多重继承和方法解析顺序
任何实现多重继承的语言都要处理潜在的命名冲突，这种冲突由不相关的祖先类实现同名方法引起。这种冲突称为“菱形问题”：
![title](rhombus.jpg)

In [43]:
class A:
    def ping(self):
        print('ping:', self)

class B(A):
    def pong(self):
        print('pong:', self)

class C(A):
    def pong(self):
        print('PONG:', self)

class D(B, C):
    def ping(self):
        super().ping()
        print('post-ping:', self)
    
    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)

In [49]:
d = D()
d.pong()
C.pong(d)
print(D.__mro__)
d.ping()
print('--------------')
d.pingpong()

pong: <__main__.D object at 0x000001D83AAC97F0>
PONG: <__main__.D object at 0x000001D83AAC97F0>
(<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>)
ping: <__main__.D object at 0x000001D83AAC97F0>
post-ping: <__main__.D object at 0x000001D83AAC97F0>
--------------
ping: <__main__.D object at 0x000001D83AAC97F0>
post-ping: <__main__.D object at 0x000001D83AAC97F0>
ping: <__main__.D object at 0x000001D83AAC97F0>
pong: <__main__.D object at 0x000001D83AAC97F0>
pong: <__main__.D object at 0x000001D83AAC97F0>
PONG: <__main__.D object at 0x000001D83AAC97F0>


#### 第十三章 正确重载运算符
##### 13.1 运算符重载基础
- 不能重载内置类型的运算符
- 不能新建运算符，只能重载现有的
- 某些运算符不能重载----is、and、or 和 not（不过位运算符
&、| 和 ~ 可以）

##### 13.2 一元运算符

In [10]:
from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools


class Vector:
    typecode = 'd'
    shortcut_names = 'xyzt'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vecotor({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        if isinstance(other, Vector):
            return len(self) == len(other) and all(a == b for a, b in zip(self, other))
        else:
            return NotImplemented
    
    def __hash__(self):
        hashes = map(hash, self._components)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.sqrt(sum(x ** 2 for x in self))
    
    def __neg__(self):
        return Vector(-x for x in self)
    
    def __pos__(self):
        return Vector(self)
    
    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented
    
    def __radd__(self, other):
        return self + other
    
    def __mul__(self, scalar):
        if isinstance(scalar, numbers.Real):
            return Vector(n * scalar for n in self)
        else:
            return NotImplemented
    
    def __rmul__(self, scalar):
        return self * scalar
    
    def __matmul__(self, other):
        try:
            return sum(a * b for a, b in zip(self, other))
        except TypeError:
            return NotImplemented
    
    def __rmatmul__(self, other):
        return self @ other

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        '''实现可切片'''
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
    
    def __getattr__(self, name):
        '''获取属性'''
        cls = type(self)
        
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
    
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_names:
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)
    
    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a
    
    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

In [11]:
v = Vector([3,4])
print(-v)
print(+v)

(-3.0, -4.0)
(3.0, 4.0)


In [63]:
# 算术运算符上下文的精度变化可能导致x不等于+x
import decimal

ctx = decimal.getcontext()
print(ctx.prec)
ctx.prec = 40
one_third = decimal.Decimal('1') / decimal.Decimal('3')
print(one_third)
print(one_third == + one_third)
ctx.prec = 28
print(one_third == + one_third)
print(+one_third)

28
0.3333333333333333333333333333333333333333
True
False
0.3333333333333333333333333333


In [70]:
# 医院运算符+的大奥一个新的Counter实例，但是没有零值和复制计数器
from collections import Counter

ct = Counter('abracadabra')
print(ct)
ct['r'] = -3
ct['d'] = 0
print(ct)
print(ct == +ct)

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})
Counter({'a': 5, 'b': 2, 'c': 1, 'd': 0, 'r': -3})
False


##### 13.3 重载向量加法运算符+
- \_\_add\_\_、\_\_radd\_\_
- 为了遵守鸭子类型精神，我们不能测试 other 操作数的类型，或者它
的元素的类型。我们要捕获异常，然后返回 NotImplemented。如果解
释器还未反转操作数，那么它将尝试去做。如果反向方法返回
NotImplemented，那么 Python 会抛出 TypeError，并返回一个标准的
错误消息，例如“unsupported operand type(s) for +: Vector
and str”。

In [86]:
v1 = Vector([3, 4, 5])
print(repr(v1 + (10, 20, 30)))
print(repr((10, 20, 30) + v1))

Vecotor([13.0, 24.0, 35.0])
Vecotor([13.0, 24.0, 35.0])


In [87]:
v1 + 1

TypeError: unsupported operand type(s) for +: 'Vector' and 'int'

##### 13.4 重载标量乘法运算符 *
Vector([1, 2, 3]) * x 是什么意思？如果 x 是数字，就是计算标量
积（scalar product），结果是一个新 Vector 实例，各个分量都会乘以
x——这也叫元素级乘法（elementwise multiplication）。

In [7]:
v1 = Vector([1.0, 2.0, 3.0])
print(14 * v1)
print(v1 * True)
from fractions import Fraction
print(v1 * Fraction(1, 3))
print(v1 @ [1, 2, 3])

(14.0, 28.0, 42.0)
(1.0, 2.0, 3.0)
(0.3333333333333333, 0.6666666666666666, 1.0)
14.0


##### 13.5 众多比较运算符
Python 解释器对众多比较运算符（==、!=、>、<、>=、<=）的处理与前文类似，不过在两个方面有重大区别。
- 正向和反向调用使用的是同一系列方法。例如，对 == 来说，正向和反向调用都是 \_\_eq\_\_ 方法，只是把参数对调了；而正向的 \_\_gt\_\_ 方法调用的是反向的 \_\_lt\_\_方法，并把参数对调。
- 对 == 和 != 来说，如果反向调用失败，Python 会比较对象的 ID，而不抛出 TypeError。

In [15]:
va = Vector([1.0, 2.0, 3.0])
vb = Vector(range(1, 4))
print(va == vb)
print(va == (1, 2, 3))
print(va != vb)

True
False
False


##### 13.6 增量赋值运算符

In [23]:
v1 = Vector([1, 2, 3])
v1_alias = v1
print(id(v1), id(v1_alias))
v1 += Vector([4, 5, 6])
print(v1, id(v1))
print(v1_alias)
v1 *= 11
print(v1, id(v1))

1825181429264 1825181429264
(5.0, 7.0, 9.0) 1825181429600
(1.0, 2.0, 3.0)
(55.0, 77.0, 99.0) 1825181429488


In [24]:
import abc

class Tombola(abc.ABC):
    
    @abc.abstractmethod
    def load(self, iterable):
        '''从可迭代对象中添加元素'''
    
    @abc.abstractmethod
    def pick(self):
        '''随机删除元素，然后将其返回
        如果实例为空， 这个方法应该抛出'LookupError'
        '''
    
    def loaded(self):
        '''如果至少有一个元素，返回'True', 否则返回'False'
        '''
        return bool(self.inspect())
    
    def inspect(self):
        '''返回一个有序元组，有当前元素构成
        '''
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

In [25]:
import random

class BingoCage(Tombola):
    
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
    
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
    
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    
    def __call__(self):
        self.pick()

In [26]:
# AddableBingoCage扩展BingoCage，支持+ 和 +=
import itertools


class AddbaleBingoCage(BingoCage):

    def __add__(self, other):
        if isinstance(ohter, Tombola):
            return AddableBingoCage(self.inspcet() + other.inspect())
        else:
            return NotImplemented
        
    def __iadd__(self, ohter):
        if isinstance(other, Tombola):
            other_iterable = other.inspect()
        else:
            try:
                other_iterable = iter(other)
            except TypeError:
                self_cls = type(self).__name__
                msg = 'right operand in += must be {!r} or an iterable'
                raise TypeError(msg.format(self_cls))
        self.load(other_iterable)
        return self